In [7]:
import requests

In [8]:
from bs4 import BeautifulSoup

In [9]:
import pandas as pd
import os
import re
from datetime import datetime
import pickle

Ich glaube wir können es uns einfach machen--die URL-Struktur sieht wie folgt aus:

`https://www.adac.de/rund-ums-fahrzeug/autokatalog/marken-modelle/[MARKE]/[MODELL]/[UNTERMODELL]/[ID]`

Nur einige Modelle haben dann noch den Unterpunkt `/#kosten`

In [15]:
outpath = "../out/test.csv"

## Modellliste laden

In [10]:
alle_modelle = pickle.load(open('../out/20220601-2354_modellist','rb'), encoding='utf-8')

## Download Modellinfos

In [11]:
len(alle_modelle)

141892

In [12]:
# current version
def downloadModelle(alle_modelle):
    print('Zu ladende Modelle: {}'.format(len(alle_modelle)))
    try: # der gesamte Prozess, wenn alles gut läuft
        i=0
        for modell_url in alle_modelle:
            # if i==9:
            #     raise(IndexError)
            if i%100==0:
                print('... {}'.format(i))
            i += 1
            ## Modell
            model_id = re.findall('/(\d+)/$',modell_url)[0]

            r_modell = requests.get(modell_url + '#technische-daten')
            modell_soup = BeautifulSoup(r_modell.text, 'html.parser')
            technische_daten = modell_soup.find_all('div', 
                                                    {"title" : "Technische Daten"})
            if len(technische_daten) > 0: # es scheinen ein paar Seiten ohne technische Daten dabei zu sein--diese überspringen
                tabellen = pd.read_html(modell_url)
                tabelle = pd.concat(tabellen, axis=0)
                tabelle['ADAC_id'] = model_id

                # Technische Daten in Datei schreiben
                tabelle.to_csv(outpath, index=False, mode='a', header=not (os.path.exists(outpath)) )
            else:
                with open(logpath, 'w') as l:
                    l.write(model_id)

    except IndexError: # wenn Abbruch: berichten, und mit dem Rest weitermachen
        # checken wie weit wir gekommen sind
        with open(outpath, "rb") as f:
            try:
                f.seek(-2, os.SEEK_END)
                while f.read(1) != b'\n':
                    f.seek(-2, os.SEEK_CUR)
            except OSError:
                f.seek(0)
            last_line = f.readline().decode()
            last_model = last_line.split(',')[-1].strip()
            print(last_model)
        print ("bekannter Fehler bei ID {}, Download wird neu aufgenommen".format(last_model))
        # bei welcher URL geht es weiter?
        for i,m in enumerate(alle_modelle):
            if(re.search(last_model,m)):
                break_index = i
                # weiter mit restlichen Modellen
                downloadModelle(alle_modelle[break_index:])

In [31]:
# pandas only
def downloadModelle_pd(alle_modelle):
    print('Zu ladende Modelle: {}'.format(len(alle_modelle)))
    try: # der gesamte Prozess, wenn alles gut läuft
        i=0
        for modell_url in alle_modelle:
            if i%100==0:
                print('... {}'.format(i))
            i += 1
            ## Modell
            model_id = re.findall('/(\d+)/$',modell_url)[0]
            tabellen = pd.read_html(modell_url)
            tabelle = pd.concat(tabellen, axis=0)
            tabelle['ADAC_id'] = model_id

            # Technische Daten in Datei schreiben
            tabelle[['Kategorie','Herstellerangabe','ADAC_id']
                   ].dropna(subset='Kategorie'
                    ).to_csv(outpath, index=False, mode='a', header=not (os.path.exists(outpath)))

            # return
            #return(tabelle)
    except IndexError: # wenn Abbruch: berichten, und mit dem Rest weitermachen
        # checken wie weit wir gekommen sind
        with open(outpath, "rb") as f:
            try:
                f.seek(-2, os.SEEK_END)
                while f.read(1) != b'\n':
                    f.seek(-2, os.SEEK_CUR)
            except OSError:
                f.seek(0)
            last_line = f.readline().decode()
            last_model = last_line.split(',')[-1].strip()
            print(last_model)
        print ("bekannter Fehler bei ID {}, Download wird neu aufgenommen".format(last_model))
        # bei welcher URL geht es weiter?
        for i,m in enumerate(alle_modelle):
            if(re.search(last_model,m)):
                break_index = i
                # weiter mit restlichen Modellen
                downloadModelle(alle_modelle[break_index:])

In [34]:
%time downloadModelle(alle_modelle[:100])

Zu ladende Modelle: 100
... 0
CPU times: user 11.1 s, sys: 286 ms, total: 11.4 s
Wall time: 54 s


In [33]:
%time downloadModelle_pd(alle_modelle[:100])

Zu ladende Modelle: 100
... 0
CPU times: user 4.05 s, sys: 149 ms, total: 4.2 s
Wall time: 29.7 s


In [27]:
downloadModelle_pd(alle_modelle[20:21])

Zu ladende Modelle: 1
... 0


,Kategorie,Herstellerangabe,ADAC_id
0,Marke,VW,282366
1,Modell,411,282366
2,Baureihe,411 Limousine (08/68 - 08/72),282366
3,Herstellerinterne Baureihenbezeichnung,411,282366
4,Modellstart,08/68,282366
...,...,...,...
2,Nebelscheinwerfer,Paket,282366
0,Grundpreis,4136 Euro,282366
1,KlassenÃ¼bliche Ausstattung nach ADAC-Vorgabe,k.A.,282366
2,Radio,Aufpreis (noch nicht bekannt),282366


In [20]:
import itertools

modell_url = alle_modelle[5]

model_id = re.findall('/(\d+)/$',modell_url)[0]
            
r_modell = requests.get(modell_url + '#technische-daten')

modell_soup = BeautifulSoup(r_modell.text, 'html.parser')

tabellen = pd.read_html(modell_url)

tabelle = pd.concat(tabellen, axis=0)

tabelle['ADAC_id'] = model_id

tabelle

# technische_daten = modell_soup.find_all('div', 
#                                         {"title" : "Technische Daten"})

# if len(technische_daten) > 0: # es scheinen ein paar Seiten ohne technische Daten dabei zu sein--diese überspringen
#     tabellen = technische_daten[0].find_all('table')
#     for t in tabellen:
#         print(pd.read_html(t))
#     # tabellen = [pd.read_html(t) if not (t is None) else None for t in tabellen]
#     ## Daten zusammensetzen

,Kategorie,Herstellerangabe,ADAC_id
0,Marke,VW,282385
1,Modell,1600,282385
2,Typ,TL,282385
3,Baureihe,1600 (Typ 3) SchrÃ¤gheck (08/65 - 08/69),282385
4,Herstellerinterne Baureihenbezeichnung,Typ 3,282385
...,...,...,...
2,Nebelscheinwerfer,Paket,282385
0,Grundpreis,3615 Euro,282385
1,KlassenÃ¼bliche Ausstattung nach ADAC-Vorgabe,k.A.,282385
2,Radio,Aufpreis (noch nicht bekannt),282385


Recovery

In [8]:
outpath = "../out/20220602-1824_adac_data.csv"
with open(outpath, "rb") as file:
    try:
        file.seek(-2, os.SEEK_END)
        while file.read(1) != b'\n':
            file.seek(-2, os.SEEK_CUR)
    except OSError:
        file.seek(0)
    last_line = file.readline().decode()
    last_model = last_line.split(',')[-1].strip()

In [47]:
last_model

'312262'

In [20]:
for i,m in enumerate(alle_modelle):
    # print(m)
    if(re.search(last_model,m)):
        this_index = i

In [30]:
this_index

88064

In [ ]:
downloadModelle(alle_modelle[this_index:])

Zu ladende Modelle: 53828
... 0
... 100
... 200
... 300
... 400
... 500
... 600
... 700
... 800
... 900
... 1000
... 1100


In [122]:
modell = alle_modelle[break_index]
## Modell
r_modell = requests.get(modell_url + '#technische-daten')

modell_soup = BeautifulSoup(r_modell.text, 'html.parser')

technische_daten = modell_soup.find_all('div', 
                                       {"title" : "Technische Daten"})
if len(technische_daten)>0:
    tabellen = technische_daten[0].find_all('table')

    ## Daten zusammensetzen
    model_id = re.findall('/(\d+)/$',modell_url)[0]

    # Tabellen parsen
    df = pd.DataFrame()
    for t in tabellen:
        kategorien = [kategorie.text for kategorie in t.find_all('td', class_='sc-hDzdEj kyOEig')]
        werte = [wert.text for wert in t.find_all('td', class_='sc-hDzdEj cGvlQj')]
        df = pd.concat([df, pd.DataFrame({'Kategorie': kategorien, 'Wert':werte})])
    df['ADAC_id'] = model_id
else:
    pass

In [123]:
technische_daten

[<div class="sc-kgwpUT elYZLd" title="Technische Daten"><main class="sc-hrJsxi iNFSzo"><div class="sc-gsFzgR gJFhVJ sc-cAhXWc jXueoB sc-gqtqkP eWGAvb" data-testid="grid-item"><div class="sc-iztaBw cLpCvt"><div class="sc-dhCHPp faCwnu"><div class="sc-igXgud ccpZNE sc-itWPBs fUUgPV"><picture class="sc-gHjyzD czgTrL" data-testid="technical-car-image"><source media="(min-width: 0px)" sizes="410" srcset="https://assets.adac.de/image/upload/ar_16:9,c_fill,f_auto,g_auto,q_auto:eco,w_410/v1/Autodatenbank/Fahrzeugbilder/im04834-1-hyundai-i10.jpg 410w, https://assets.adac.de/image/upload/ar_16:9,c_fill,f_auto,g_auto,q_auto:eco,w_820/v1/Autodatenbank/Fahrzeugbilder/im04834-1-hyundai-i10.jpg 820w,"/><img alt="i10 1.2" class="sc-JEhMO dKwzVf" data-src="https://assets.adac.de/image/upload/ar_16:9,c_fill,f_auto,g_auto,q_auto:eco,w_410/v1/Autodatenbank/Fahrzeugbilder/im04834-1-hyundai-i10.jpg" loading="lazy" title="i10 1.2"/></picture></div></div><div class="sc-kxIisj eRAhYE"><div class="sc-eAcRFu bYg

In [101]:
modell = alle_modelle[break_index-10]
## Modell
r_modell = requests.get(modell_url + '#technische-daten')

modell_soup = BeautifulSoup(r_modell.text, 'html.parser')

technische_daten = modell_soup.find_all('div', 
                                        class_ = 'sc-cxVPaa gxVuRJ sc-bnOPBZ jbcYSh sc-iKMXQg iMhTHO')
tabellen = technische_daten[0].find_all('table')

## Daten zusammensetzen
model_id = re.findall('/(\d+)/$',modell_url)[0]

# Tabellen parsen
df = pd.DataFrame()
for t in tabellen:
    kategorien = [kategorie.text for kategorie in t.find_all('td', class_='sc-hDzdEj kyOEig')]
    werte = [wert.text for wert in t.find_all('td', class_='sc-hDzdEj cGvlQj')]
    df = pd.concat([df, pd.DataFrame({'Kategorie': kategorien, 'Wert':werte})])
df['ADAC_id'] = model_id

IndexError: list index out of range

In [103]:
divs = modell_soup.find_all('div')

In [104]:
len(divs)

405

In [107]:
[d.get_title() for d in divs]

TypeError: 'NoneType' object is not callable